# Understanding Regularization in Polynomial Regression

## The Overfitting Problem

When you have a high-degree polynomial, the model can become extremely flexible and starts "chasing" individual data points, especially where data is sparse (like between those last two points). This creates:
- **High variance** between data points
- **Poor generalization** to new data
- **Erratic behavior** in cross-validation

## What is Regularization?

Regularization adds a "penalty term" to discourage overly complex models. Instead of just minimizing the fit error, you minimize:

**Original (unregularized):**
$$(Y - M\theta)^T C^{-1} (Y - M\theta)$$
This is just "how well do I fit the data?"

**Regularized (Ridge Regression):**
$$(Y - M \theta)^T C^{-1} (Y- M \theta) + \lambda\, \theta^T\theta$$
This is "how well do I fit the data + **how large are my coefficients?**"

## Understanding the Terms

1. **First term**: $(Y - M\theta)^T C^{-1} (Y - M\theta)$ 
   - Measures fit quality (smaller = better fit)
   - This is your weighted sum of squared residuals

2. **Second term**: $\lambda\, \theta^T\theta$
   - Penalty for large coefficients (smaller coefficients = simpler model)
   - $\theta^T\theta = \theta_1^2 + \theta_2^2 + ... + \theta_n^2$ (sum of squared coefficients)
   - $\lambda$ controls how much you care about this penalty

## The Regularization Parameter λ

- **λ = 0**: No penalty → can overfit
- **λ small**: Gentle penalty → allows some flexibility
- **λ large**: Strong penalty → forces simpler model (may underfit)
- **λ → ∞**: All coefficients forced to ~0 → extreme underfitting

## The Solution

The regularized solution becomes:
$$\hat\theta = (M^T C^{-1} M + \lambda I)^{-1} (M^T C^{-1} Y)$$

Notice the **$+ \lambda I$** term - this is the key difference! It:
- Makes the matrix inversion more stable
- Shrinks coefficients toward zero
- Prevents any single coefficient from becoming too large

## Bayesian Interpretation

From a Bayesian view, adding $\lambda\, \theta^T\theta$ is equivalent to saying:

**"I believe a priori that coefficients should be small"**

The prior is:
$$p(\theta | I ) \propto \exp{\left(\frac{-\lambda\, \theta^T\theta}{2}\right)}$$

This is a **Gaussian prior centered at zero** - you're saying "I expect coefficients near zero, and I'm increasingly surprised by larger values."

## Why It Works

High-degree polynomials need **large coefficients** to create those wild oscillations between data points. By penalizing large coefficients, regularization:

1. **Smooths the function** - prevents wild swings
2. **Reduces variance** - more stable predictions
3. **Improves generalization** - better cross-validation performance

## The Sweet Spot - Bias-Variance Tradeoff

The goal is to find the balance between:
- **Underfit** (λ too large): High bias, low variance - model too simple
- **Balanced** (λ optimal): Good bias-variance tradeoff
- **Overfit** (λ = 0): Low bias, high variance - model too complex

The optimal λ minimizes test/validation error, giving you the best generalization to new data.

---

**In your cosmology context**: Even if you use a degree-5 polynomial, regularization can prevent those crazy oscillations between your last data points by keeping the high-order coefficients small!

# Least Absolute Shrinkage and Selection (LASSO) Regularization

## What is LASSO?

An alternative to Ridge Regression is **LASSO**, which uses a different penalty term:

**LASSO regularization:**
$$(Y - M \theta)^T(Y- M \theta) + \lambda \sum_i|\theta_i|$$

This is equivalent to least-squares minimization with the restriction that:
$$ \sum_i|\theta_i| < s$$

## Key Difference from Ridge Regression

**Ridge**: Penalty on the sum of **squared** coefficients → $\lambda\, \theta^T\theta = \lambda \sum_i \theta_i^2$

**LASSO**: Penalty on the sum of **absolute values** → $\lambda \sum_i|\theta_i|$

## The Power of L1 Regularization

The absolute value penalty has a special property: it **preferentially selects regions of likelihood space that coincide with one of the vertices** within the region defined by the regularization.

### What does this mean?

***LASSO has the net effect of setting one (or more) of the model coefficients to exactly zero.***

## LASSO vs Ridge: Feature Selection

| Property | Ridge Regression | LASSO |
|----------|------------------|-------|
| Penalty | $\sum_i \theta_i^2$ | $\sum_i \|\theta_i\|$ |
| Coefficient behavior | Shrinks toward zero | Sets some to **exactly zero** |
| Feature selection | No (keeps all features) | Yes (automatic feature selection) |
| Sparsity | No | Yes |

## Why LASSO Creates Sparsity

The L1 penalty (absolute value) creates "corners" in the constraint region. When the likelihood contours intersect these corners, coefficients hit exactly zero.

**Ridge** (L2): Smooth circular/elliptical constraint → coefficients get small but rarely exactly zero

**LASSO** (L1): Diamond-shaped constraint with corners → coefficients often hit exactly zero at the corners

## Practical Implications

1. **Automatic feature selection**: LASSO identifies which polynomial terms are truly necessary
2. **Interpretability**: Simpler models with fewer non-zero coefficients
3. **Sparsity**: Removes irrelevant features entirely

## When to Use LASSO vs Ridge

- **Use LASSO when**: 
  - You suspect only some features are important
  - You want automatic feature selection
  - You prefer a sparse, interpretable model

- **Use Ridge when**:
  - You believe most features contribute
  - You want to shrink all coefficients smoothly
  - Coefficients are highly correlated

---

**In your polynomial regression**: LASSO might decide that only degrees 1, 2, and 4 matter, setting the coefficients for degrees 3 and 5 to exactly zero!

# Gaussian Process Kernel Selection Guide

## Understanding Kernel Choice

Different kernels encode different assumptions about your function. For **cosmological distance modulus vs redshift**, here's what you should consider:

---

## Recommended Kernels for Your Data

### 1. **Matérn Kernel** (BEST for cosmology ⭐)

```python
from sklearn.gaussian_process.kernels import Matern

kernel = ConstantKernel(1.0) * Matern(length_scale=1.0, nu=2.5)
```

**Why it's good:**
- ✅ Flexible smoothness control via `nu` parameter
- ✅ Not infinitely smooth (more realistic for real data)
- ✅ Works well with noisy measurements
- ✅ **Standard choice in cosmology and astrophysics**

**The `nu` parameter:**
- `nu = 0.5`: Very rough (equivalent to Exponential kernel)
- `nu = 1.5`: Once differentiable (**good default**)
- `nu = 2.5`: Twice differentiable (**smooth, good for cosmology**)
- `nu → ∞`: Approaches RBF (infinitely smooth)

**For distance modulus:** Use `nu = 2.5` or `nu = 1.5`

---

### 2. **RBF (Radial Basis Function)** - Also called Squared Exponential

```python
from sklearn.gaussian_process.kernels import RBF

kernel = ConstantKernel(1.0) * RBF(length_scale=10.0)
```

**Why it's good:**
- ✅ Produces very smooth functions
- ✅ Good for continuous, smooth cosmological relations
- ✅ Simple - only one hyperparameter (length_scale)

**Why it might be too simple:**
- ⚠️ Infinitely differentiable (too smooth for real data?)
- ⚠️ Can't capture sharp transitions
- ⚠️ May oversmooth noisy regions

**For distance modulus:** Good if you expect perfectly smooth cosmological curve

---

### 3. **Rational Quadratic** (Good alternative)

```python
from sklearn.gaussian_process.kernels import RationalQuadratic

kernel = ConstantKernel(1.0) * RationalQuadratic(length_scale=1.0, alpha=1.0)
```

**Why it's good:**
- ✅ Mixture of RBF kernels with different length scales
- ✅ More flexible than RBF
- ✅ Can capture multi-scale structure

**The `alpha` parameter:**
- Controls the relative weighting of large-scale vs small-scale variations
- Larger `alpha` → more like RBF (single scale)
- Smaller `alpha` → captures multiple scales

**For distance modulus:** Good if data has features at different scales

---

### 4. **Exponential Sine Squared** (For periodic features)

```python
from sklearn.gaussian_process.kernels import ExpSineSquared

kernel = ConstantKernel(1.0) * ExpSineSquared(length_scale=1.0, periodicity=1.0)
```

**When to use:**
- ❌ **NOT for distance modulus** (not periodic!)
- ✅ Good for: stellar light curves, climate data, seasonal patterns

---

### 5. **Composite Kernels** (Most flexible ⭐⭐)

You can **combine** kernels with `+` and `*` operators!

#### Example 1: Smooth trend + small-scale noise
```python
# Long-term smooth trend + short-term variations
kernel = (ConstantKernel(1.0) * Matern(length_scale=10.0, nu=2.5) +
          ConstantKernel(0.1) * Matern(length_scale=0.5, nu=1.5))
```

#### Example 2: Multiplicative (for non-stationary data)
```python
# Amplitude varies with redshift
kernel = (ConstantKernel(1.0) * RBF(length_scale=10.0) * 
          RBF(length_scale=1.0))
```

#### Example 3: Additive (for multiple components)
```python
# Main signal + noise
kernel = (ConstantKernel(10.0) * Matern(length_scale=5.0, nu=2.5) +
          WhiteKernel(noise_level=0.1))
```

---

## My Specific Recommendations for Distance Modulus

### **Option 1: Simple and Robust** (Start here!)
```python
from sklearn.gaussian_process.kernels import ConstantKernel, Matern

kernel = ConstantKernel(1.0, constant_value_bounds=(0.1, 100)) * \
         Matern(length_scale=5.0, length_scale_bounds=(0.1, 50), nu=2.5)
```

**Why:** Smooth enough for cosmology, but not over-smooth. Standard choice.

---

### **Option 2: More Flexible** (If Option 1 underfits)
```python
kernel = ConstantKernel(1.0, constant_value_bounds=(0.1, 100)) * \
         Matern(length_scale=5.0, length_scale_bounds=(0.1, 50), nu=1.5)
```

**Why:** `nu=1.5` is less smooth than `nu=2.5`, can capture more detail.

---

### **Option 3: Multi-scale** (If you see structure at different scales)
```python
# Long-range smooth component + short-range details
kernel = (ConstantKernel(5.0) * Matern(length_scale=10.0, nu=2.5) +
          ConstantKernel(1.0) * Matern(length_scale=1.0, nu=1.5))
```

**Why:** Captures both the overall cosmological trend AND local variations.

---

### **Option 4: Let sklearn optimize** (Automated!)
```python
# Use optimizable bounds and let GP find best hyperparameters
kernel = ConstantKernel(1.0, constant_value_bounds=(1e-3, 1e3)) * \
         Matern(length_scale=1.0, length_scale_bounds=(1e-2, 1e2), nu=1.5)

gp = GaussianProcessRegressor(kernel=kernel, alpha=dmu**2, 
                               n_restarts_optimizer=10)
gp.fit(z_sample.reshape(-1, 1), mu_sample)

print(f"Optimized kernel: {gp.kernel_}")
```

**Why:** GP automatically finds best hyperparameters via maximum likelihood. Set `n_restarts_optimizer=10` to avoid local minima.

---

## Kernel Comparison Table

| Kernel | Smoothness | Flexibility | Cosmology? | Parameters |
|--------|-----------|-------------|------------|------------|
| **RBF** | ∞ smooth | Low | ✅ Good | `length_scale` |
| **Matérn (nu=2.5)** | Smooth | Medium | ⭐ Best | `length_scale, nu` |
| **Matérn (nu=1.5)** | Moderate | High | ✅ Good | `length_scale, nu` |
| **Rational Quadratic** | Variable | High | ✅ Good | `length_scale, alpha` |
| **Exponential** | Rough | Low | ❌ No | `length_scale` |
| **Composite** | Custom | Very High | ⭐ Best | Multiple |

---

## Quick Decision Tree

```
Do you know the smoothness level?
├─ Yes, very smooth → Use RBF
└─ No / unsure
   ├─ Single scale features → Use Matérn (nu=2.5)
   └─ Multiple scale features → Use composite kernel

Is your model underfitting?
├─ Yes → Decrease length_scale or use nu=1.5
└─ No → Good!

Is your model overfitting?
├─ Yes → Increase length_scale or use nu=2.5
└─ No → Good!
```

---

## Code to Test Multiple Kernels

```python
from sklearn.gaussian_process.kernels import *

kernels = {
    'RBF': ConstantKernel(1.0) * RBF(length_scale=5.0),
    'Matérn-1.5': ConstantKernel(1.0) * Matern(length_scale=5.0, nu=1.5),
    'Matérn-2.5': ConstantKernel(1.0) * Matern(length_scale=5.0, nu=2.5),
    'RatQuad': ConstantKernel(1.0) * RationalQuadratic(length_scale=5.0, alpha=1.0),
    'Multi-scale': (ConstantKernel(5.0) * Matern(length_scale=10.0, nu=2.5) +
                    ConstantKernel(1.0) * Matern(length_scale=1.0, nu=1.5))
}

for name, kernel in kernels.items():
    gp = GaussianProcessRegressor(kernel=kernel, alpha=dmu**2)
    scores = cross_val_score(gp, z_sample.reshape(-1, 1), mu_sample, 
                            cv=5, scoring='neg_mean_squared_error')
    rmse = np.sqrt(-scores.mean())
    print(f"{name:15s}: RMSE = {rmse:.4f} ± {np.sqrt(scores.std()):.4f}")
```

---

## TL;DR - Just tell me what to use!

**For distance modulus vs redshift:**

```python
kernel = ConstantKernel(1.0) * Matern(length_scale=5.0, nu=2.5)
```

Then use GridSearchCV to optimize the hyperparameters. Done! 🎉

If that underfits, try `nu=1.5` for more flexibility.
If it overfits, try RBF for maximum smoothness.

# Parametric vs Data-Driven Fits: A Complete Guide

## Quick Definition

| Aspect | Parametric | Data-Driven |
|--------|-----------|-------------|
| **Based on** | Physical theory/model | Data patterns only |
| **Parameters** | Few (2-5), interpretable | Many, often not interpretable |
| **Flexibility** | Limited by model | High - adapts to data |
| **Extrapolation** | Good (theory-guided) | Poor (no theory) |
| **Example** | ΛCDM cosmology | Gaussian Process |

---

## Detailed Explanation

### Parametric Fit

**Definition:** Fitting assumes a **specific functional form** derived from physical theory or mathematical model.

**Key characteristics:**
1. ✅ **Few parameters** - typically 2-10
2. ✅ **Interpretable** - parameters have physical meaning
3. ✅ **Theory-driven** - functional form from first principles
4. ✅ **Extrapolates well** - physics guides behavior outside data range
5. ❌ **Can be wrong** - if model assumptions violated
6. ❌ **Inflexible** - can't capture unexpected features

**Mathematical form:**
$$y = f(x; \theta_1, \theta_2, ..., \theta_n)$$

Where:
- f is a **known function** (from theory)
- θ₁, θ₂, ... are the **parameters to fit**
- n is **small** (usually < 10)

---

### Data-Driven Fit (Non-Parametric)

**Definition:** Fitting makes **minimal assumptions** about functional form. Lets data determine the shape.

**Key characteristics:**
1. ✅ **Flexible** - adapts to any smooth pattern
2. ✅ **No model assumptions** - doesn't require theory
3. ✅ **Discovers unexpected features** - can reveal new physics
4. ✅ **Model-independent** - unbiased by theory
5. ❌ **Many hyperparameters** - kernel choice, length scales, etc.
6. ❌ **Poor extrapolation** - no guidance outside data
7. ❌ **Less interpretable** - hard to extract physical meaning

**Mathematical form:**
$$y = f(x) \text{ where } f \text{ is learned from data}$$

Where:
- f is **not specified in advance**
- Shape determined by data + smoothness assumptions
- Infinite-dimensional (not reducible to few parameters)

---

## Concrete Example: Supernova Distance Modulus

### Your Data: μ(z) - Distance modulus vs redshift

---

### Parametric Approach: ΛCDM Model

**Functional form from General Relativity:**
$$\mu(z) = 5 \log_{10} \left( \frac{c}{H_0 \cdot 10\text{pc}} (1+z) \int_0^z \frac{dz'}{\sqrt{\Omega_m (1+z')^3 + \Omega_\Lambda}} \right)$$

**Parameters to fit:**
- **H₀** = Hubble constant (expansion rate today)
- **Ωₘ** = matter density parameter

**Only 2 parameters!**

**What you're doing:**
- ✅ Using Einstein's equations
- ✅ Assuming flat universe (Ωₘ + ΩΛ = 1)
- ✅ Assuming dark energy is cosmological constant
- ✅ Getting physically meaningful results

**Advantages:**
```python
# Simple, interpretable
H0 = 70.0  # km/s/Mpc - has units, physical meaning
Omega_m = 0.3  # dimensionless - fraction of universe that's matter
```

**Can answer:**
- ✓ What is the Hubble constant?
- ✓ How much matter vs dark energy?
- ✓ Will universe expand forever?
- ✓ What happens at z = 10 (extrapolation)?

---

### Data-Driven Approach: Gaussian Process

**Functional form:**
$$\mu(z) \sim \mathcal{GP}(m(z), k(z, z'))$$

No explicit equation! GP learns the function from data.

**"Parameters" (actually hyperparameters):**
- Kernel choice (RBF, Matérn, etc.)
- Constant amplitude (σ²)
- Length scale (ℓ)
- Smoothness parameter (ν for Matérn)

**Many effective parameters** (infinite-dimensional function space)

**What you're doing:**
- ✅ Assuming smoothness (via kernel)
- ✅ Letting data determine shape
- ❌ NOT using physical theory
- ❌ Parameters have no cosmological meaning

**Advantages:**
```python
# Flexible, no theory needed
length_scale = 5.3  # correlation length - no physical units
constant = 12.7     # amplitude - just a scale factor
# What do these mean physically? Nothing specific!
```

**Can answer:**
- ✓ What is μ at z = 0.5 (interpolation)?
- ✓ How uncertain is the prediction?
- ✓ Are there unexpected features?
- ✗ What is H₀? (Can't extract directly)
- ✗ What happens at z = 10? (Unreliable extrapolation)

---

## Side-by-Side Comparison

### Example: Polynomial Regression

#### Parametric: Fixed degree polynomial
```python
# Assume quadratic relationship (theory suggests this)
def model(x, a, b, c):
    return a*x**2 + b*x + c

# Fit: find 3 parameters
params, _ = curve_fit(model, x, y)
```

**Properties:**
- 3 parameters only
- Assumes exactly quadratic (could be wrong!)
- Extrapolates as parabola

#### Data-Driven: Flexible polynomial degree
```python
# Try many polynomial degrees, pick best via cross-validation
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge

# Let data choose complexity
for degree in range(1, 15):
    model = Ridge()
    cv_score = cross_val_score(model, X_poly, y)
    # Pick degree with best score
```

**Properties:**
- Many parameters (one per term)
- No assumption about "true" degree
- Cross-validation chooses complexity
- May overfit or underfit

---

## When to Use Each

### Use Parametric When:
1. ✅ **You have a physical model** - theory predicts functional form
2. ✅ **Interpretability matters** - need physically meaningful parameters
3. ✅ **Extrapolation needed** - must predict outside data range
4. ✅ **Small dataset** - few data points, need constraints
5. ✅ **Publication/communication** - easier to explain and justify

**Examples:**
- Cosmological distance-redshift relation (ΛCDM)
- Stellar evolution models
- Orbital mechanics (Kepler's laws)
- Exponential decay (radioactivity)
- Power laws (scaling relations)

### Use Data-Driven When:
1. ✅ **No theory available** - exploring unknown territory
2. ✅ **Theory might be wrong** - want unbiased look at data
3. ✅ **Complex/unknown functional form** - too complicated to model
4. ✅ **Interpolation only** - don't need to extrapolate
5. ✅ **Large dataset** - enough data to constrain flexible model
6. ✅ **Exploratory analysis** - discovering new patterns

**Examples:**
- Machine learning predictions
- Detrending/smoothing noisy data
- Anomaly detection
- Image recognition
- Speech recognition

---

## Hybrid Approaches

Sometimes you combine both!

### Example 1: Parametric + Gaussian Process Residuals
```python
# Parametric: main signal
mu_theory = LCDM_model(z, H0, Omega_m)

# Data-driven: capture deviations
residuals = mu_obs - mu_theory
gp = GaussianProcessRegressor()
gp.fit(z, residuals)

# Final prediction
mu_final = mu_theory + gp.predict(z_new)
```

**Use case:** ΛCDM is mostly right, but there are small deviations (maybe new physics!)

### Example 2: Physics-Informed Neural Networks
```python
# Neural network (data-driven) 
# but loss function includes physics equations (parametric constraints)
loss = data_loss + lambda * physics_loss
```

**Use case:** Complex system where exact solution unknown, but physics provides constraints

---

## Philosophy: Two Ways of Knowing

### Parametric (Deductive)
**"Top-down" reasoning:**
1. Start with theory (General Relativity)
2. Derive predictions (μ(z) formula)
3. Fit parameters to data (H₀, Ωₘ)
4. Confirm or reject theory

**Example reasoning:**
> "Einstein's equations predict this relationship. If the universe contains matter and dark energy in these proportions, we should see this distance-redshift curve."

### Data-Driven (Inductive)
**"Bottom-up" reasoning:**
1. Start with data
2. Find patterns
3. Build empirical model
4. (Maybe) develop theory later

**Example reasoning:**
> "The data shows this smooth curve. I don't know why, but I can predict intermediate values accurately."

---

## Practical Tips

### For Your Homework/Analysis:

**First:** Do data-driven (GP) analysis
- ✅ Explore the data without bias
- ✅ Check for anomalies or unexpected features
- ✅ Establish that data is smooth and well-behaved

**Then:** Do parametric (ΛCDM) fit
- ✅ Test physical model
- ✅ Extract meaningful parameters (H₀, Ωₘ)
- ✅ Compare to theory predictions

**Finally:** Compare approaches
- Do residuals (data - ΛCDM) show patterns?
- Does GP capture something ΛCDM misses?
- Is ΛCDM adequate or do we need new physics?

---

## Common Misconceptions

### ❌ "Parametric means linear regression"
**Reality:** Parametric just means fixed functional form. Can be highly nonlinear (like ΛCDM integral!)

### ❌ "Data-driven is always better because it's more flexible"
**Reality:** Flexibility can lead to overfitting. Without theory, you might fit noise.

### ❌ "GP has no parameters"
**Reality:** GP has hyperparameters (kernel, length scales). They're just not the "parameters of interest" like H₀.

### ❌ "Parametric models can't be wrong"
**Reality:** Theory-based models can absolutely be wrong! That's why we test them.

### ❌ "You must choose one approach"
**Reality:** Use both! They provide complementary information.

---

## Real-World Analogy

### Parametric: Following a Recipe
- Recipe says: "Bake at 180°C for 30 minutes"
- Based on theory: chemistry of baking
- Parameters: temperature, time
- If you follow recipe at 200°C for 60 min, you can predict (burnt cake)

### Data-Driven: Tasting and Adjusting
- Try different amounts, taste, adjust
- No recipe, just experience
- Learn empirically: "More sugar → sweeter"
- Can't predict what happens with caviar (outside training data)

**Best approach:** Use recipe (theory) as starting point, but adjust based on tasting (data)!

---

## Summary Table

| Feature | Parametric | Data-Driven |
|---------|-----------|-------------|
| **Basis** | Physical theory | Data patterns |
| **# Parameters** | Few (2-10) | Many (or infinite) |
| **Interpretability** | High | Low |
| **Flexibility** | Low | High |
| **Extrapolation** | Good | Poor |
| **Overfitting risk** | Low | High |
| **Requires theory** | Yes | No |
| **Best for** | Well-understood physics | Exploratory analysis |
| **Your SN example** | ΛCDM (H₀, Ωₘ) | Gaussian Process |

---

## The Bottom Line

**Parametric:** 
> "I know the physics, I just need to measure the parameters."

**Data-Driven:** 
> "I don't know the physics, let the data speak."

**Best Practice:**
> "Use data-driven to explore, parametric to explain."

For cosmology: Start with GP to ensure no surprises, then fit ΛCDM to extract physical parameters!

🎯 **Remember:** Neither is "better" - they answer different questions!